In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os

### Load air_quality datasets

In [2]:
df_contaminants = pd.read_csv("../data/raw/qualitat-aire-contaminants/qualitat_aire_contaminants.csv")
df_contaminants['Contaminant_Units'] = df_contaminants['Desc_Contaminant'] + ' (' + df_contaminants['Unitats'] + ')'
df_contaminants.drop(columns=['Desc_Contaminant', 'Unitats'], inplace=True)
df_contaminants.head(50)

,Codi_Contaminant,Contaminant_Units
0,1,SO2 (µg/m³)
1,6,CO (mg/m³)
2,7,NO (µg/m³)
3,8,NO2 (µg/m³)
4,9,PM2.5 (µg/m³)
5,10,PM10 (µg/m³)
6,12,NOx (µg/m³)
7,14,O3 (µg/m³)
8,22,Black Carbon (µg/m³)
9,101,SO2* (µg/m³)


In [3]:
df_contaminants['Contaminant_Units'].unique()

array(['SO2 (µg/m³)', 'CO (mg/m³)', 'NO (µg/m³)', 'NO2 (µg/m³)',
       'PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NOx (µg/m³)', 'O3 (µg/m³)',
       'Black Carbon (µg/m³)', 'SO2* (µg/m³)', 'CO* (mg/m³)',
       'NO* (µg/m³)', 'NO2* (µg/m³)', 'PM2.5* (µg/m³)', 'PM10* (µg/m³)',
       'Nox* (µg/m³)', 'O3* (µg/m³)', nan,
       'Flow C (Mesura interna equip)  ( )', 'Biomassa Black Carbon (%)'],
      dtype=object)

In [4]:
df_stations = pd.read_csv("../data/raw/qualitat-aire-estacions-bcn/2019_Qualitat_Aire_Estacions.csv")
print(df_stations.columns)
df_stations = df_stations[['Estacio', 'Nom_districte', 'Longitud', 'Latitud']].sort_values(by='Estacio').drop_duplicates()
df_stations.head()

Index(['Estacio', 'nom_cabina', 'codi_dtes', 'zqa', 'codi_eoi', 'Longitud',
       'Latitud', 'ubicacio', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Clas_1', 'Clas_2', 'Codi_Contaminant'],
      dtype='object')


,Estacio,Nom_districte,Longitud,Latitud
27,4,Sant Marti,2.2045,41.4039
29,42,Sants-Montjuic,2.1331,41.3788
4,43,Eixample,2.1538,41.3853
17,44,Gracia,2.1534,41.3987
3,50,Ciutat Vella,2.1874,41.3864


In [5]:
def load_and_concat_csvs(folder_path):
    # Get all CSV files in the folder
    csv_files = []
    for file in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file)
        if os.path.isfile(full_path) and file.endswith(".csv"):
            csv_files.append(full_path)
    
    # Read files (skip headers after the first file)
    df_list = []
    for i, file in enumerate(csv_files):
        df = pd.read_csv(file)
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

# Usage
folder_path = "../data/raw/qualitat-aire-detall-bcn"
df_air = load_and_concat_csvs(folder_path)
df_air.head()

,CODI_PROVINCIA,PROVINCIA,CODI_MUNICIPI,MUNICIPI,ESTACIO,CODI_CONTAMINANT,ANY,MES,DIA,H01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,8,Barcelona,19,Barcelona,4,7,2019,4,2,1.0,...,8.0,V,3.0,V,1.0,V,1.0,V,1.0,V
1,8,Barcelona,19,Barcelona,4,7,2019,4,3,1.0,...,3.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,8,Barcelona,19,Barcelona,4,7,2019,4,4,1.0,...,2.0,V,2.0,V,1.0,V,2.0,V,2.0,V
3,8,Barcelona,19,Barcelona,4,7,2019,4,5,2.0,...,3.0,V,2.0,V,2.0,V,1.0,V,2.0,V
4,8,Barcelona,19,Barcelona,4,7,2019,4,6,1.0,...,6.0,V,2.0,V,2.0,V,1.0,V,3.0,V


In [6]:
values_to_remove = [996, 997, 998, 999]
duplicated_values = [101, 106, 107, 108, 109, 110, 112, 114]

df_air.drop(df_air[df_air['CODI_CONTAMINANT'].isin(values_to_remove)].index, inplace=True)
df_air['CODI_CONTAMINANT'] = df_air['CODI_CONTAMINANT'].apply(lambda x: x-100 if x in duplicated_values else x)

### Merge dataframes

In [7]:
# Melt the hourly columns into rows
df_melted = df_air.melt(
    id_vars=['ESTACIO', 'CODI_CONTAMINANT', 'ANY', 'MES', 'DIA'],
    value_vars=[f'H{i:02d}' for i in range(1, 25)],
    var_name='HourColumn',
    value_name='Value'
)

# Convert hour code to integer (H01=1 -> 01:00, H24=24 -> next day's 00:00)
df_melted['hour'] = df_melted['HourColumn'].str[1:].astype(int)

# Create base date from components
df_melted['base_date'] = pd.to_datetime(
    df_melted['ANY'].astype(str) + '-' +
    df_melted['MES'].astype(str).str.zfill(2) + '-' +
    df_melted['DIA'].astype(str).str.zfill(2)
)

# Add hours as timedelta (handles H24 automatically)
df_melted['timestamp'] = df_melted['base_date'] + pd.to_timedelta(df_melted['hour'], unit='h')

# Select final columns
result = df_melted[['timestamp', 'ESTACIO', 'CODI_CONTAMINANT', 'Value']]\
           .sort_values('timestamp')\
           .reset_index(drop=True)

df_air_clean = result
df_air_clean.head()

,timestamp,ESTACIO,CODI_CONTAMINANT,Value
0,2019-04-02 01:00:00,4,7,1.0
1,2019-04-02 01:00:00,44,8,56.0
2,2019-04-02 01:00:00,44,7,13.0
3,2019-04-02 01:00:00,44,6,0.3
4,2019-04-02 01:00:00,44,1,1.0


In [8]:
df_merged = df_air_clean.merge(df_stations, left_on='ESTACIO', right_on='Estacio', how='left')
df_merged = df_merged.merge(df_contaminants, left_on='CODI_CONTAMINANT', right_on='Codi_Contaminant', how='left')
df_merged.drop(columns=['ESTACIO', 'CODI_CONTAMINANT'], inplace=True)
df_merged.sort_values(by=['timestamp', 'Estacio', 'Nom_districte'])
df_merged.head(10)

,timestamp,Value,Estacio,Nom_districte,Longitud,Latitud,Codi_Contaminant,Contaminant_Units
0,2019-04-02 01:00:00,1.0,4.0,Sant Marti,2.2045,41.4039,7.0,NO (µg/m³)
1,2019-04-02 01:00:00,56.0,44.0,Gracia,2.1534,41.3987,8.0,NO2 (µg/m³)
2,2019-04-02 01:00:00,13.0,44.0,Gracia,2.1534,41.3987,7.0,NO (µg/m³)
3,2019-04-02 01:00:00,0.3,44.0,Gracia,2.1534,41.3987,6.0,CO (mg/m³)
4,2019-04-02 01:00:00,1.0,44.0,Gracia,2.1534,41.3987,1.0,SO2 (µg/m³)
5,2019-04-02 01:00:00,75.0,43.0,Eixample,2.1538,41.3853,14.0,O3 (µg/m³)
6,2019-04-02 01:00:00,32.0,43.0,Eixample,2.1538,41.3853,12.0,NOx (µg/m³)
7,2019-04-02 01:00:00,15.0,43.0,Eixample,2.1538,41.3853,10.0,PM10 (µg/m³)
8,2019-04-02 01:00:00,32.0,50.0,Ciutat Vella,2.1874,41.3864,8.0,NO2 (µg/m³)
9,2019-04-02 01:00:00,27.0,43.0,Eixample,2.1538,41.3853,8.0,NO2 (µg/m³)


In [9]:
df_merged['Contaminant_Units'].unique()

array(['NO (µg/m³)', 'NO2 (µg/m³)', 'CO (mg/m³)', 'SO2 (µg/m³)',
       'O3 (µg/m³)', 'NOx (µg/m³)', 'PM10 (µg/m³)', nan,
       'Black Carbon (µg/m³)', 'PM2.5 (µg/m³)'], dtype=object)

In [10]:
# Pivot the table to create columns for each Contaminant_Units
df_pivot = df_merged.pivot_table(
    index=['timestamp', 'Estacio', 'Nom_districte', 'Longitud', 'Latitud'],
    columns='Contaminant_Units',
    values='Value',
    aggfunc='first'  
).reset_index()

# Clean up column names
df_pivot.columns.name = None  
df_pivot = df_pivot.rename_axis(columns=None)  
df_pivot.fillna(0, inplace=True)

df_pivot.head()

,timestamp,Estacio,Nom_districte,Longitud,Latitud,Black Carbon (µg/m³),CO (mg/m³),NO (µg/m³),NO2 (µg/m³),NOx (µg/m³),O3 (µg/m³),PM10 (µg/m³),PM2.5 (µg/m³),SO2 (µg/m³)
0,2019-04-02 01:00:00,4.0,Sant Marti,2.2045,41.4039,0.0,0.0,1.0,22.0,22.0,0.0,19.0,0.0,0.0
1,2019-04-02 01:00:00,42.0,Sants-Montjuic,2.1331,41.3788,0.0,0.0,2.0,29.0,32.0,0.0,0.0,0.0,0.0
2,2019-04-02 01:00:00,43.0,Eixample,2.1538,41.3853,0.0,0.2,3.0,27.0,32.0,75.0,15.0,0.0,1.0
3,2019-04-02 01:00:00,44.0,Gracia,2.1534,41.3987,0.0,0.3,13.0,56.0,76.0,44.0,21.0,0.0,1.0
4,2019-04-02 01:00:00,50.0,Ciutat Vella,2.1874,41.3864,0.0,0.0,1.0,32.0,32.0,64.0,0.0,0.0,0.0


In [11]:
# Save processed table
df_pivot.to_parquet("../data/processed/air_quality/air_quality.parquet")

### Merge Inhaler & Air Quality Tables

In [12]:
df_inhaler = pd.read_parquet("../data/raw/iot_inhaler", engine='pyarrow')
df_air_quality = pd.read_parquet("../data/processed/air_quality/air_quality.parquet", engine='pyarrow')

In [13]:
df_air_quality.rename(columns={'Nom_districte':'district',
                               'Estacio': 'station',
                               'Longitud': 'longitude',
                               'Latitud': 'latitude'},
                      inplace=True)

In [14]:
df_inhaler_columns = ['patient_id', 'timestamp', 'puffs', 'date', 'age', 'gender', 'district', 'smoker']
df_air_quality_columns = ['timestamp', 'station', 'district', 'longitude', 'latitude', 'PM2.5 (µg/m³)','NO2 (µg/m³)', 'O3 (µg/m³)']

In [15]:
df_inhaler.head()

,timestamp,puffs,pm25,no2,date,age,gender,district,smoker,patient_id
0,2022-01-01 00:00:00,0,20.4,48.9,2022-01-01,70,F,Ciutat Vella,0,PAT_0000
1,2022-01-01 01:00:00,0,26.8,47.0,2022-01-01,70,F,Ciutat Vella,0,PAT_0000
2,2022-01-01 02:00:00,0,28.8,55.2,2022-01-01,70,F,Ciutat Vella,0,PAT_0000
3,2022-01-01 03:00:00,0,26.3,39.2,2022-01-01,70,F,Ciutat Vella,0,PAT_0000
4,2022-01-01 04:00:00,0,19.5,37.9,2022-01-01,70,F,Ciutat Vella,0,PAT_0000


In [16]:
df_air_quality.head()

,timestamp,station,district,longitude,latitude,Black Carbon (µg/m³),CO (mg/m³),NO (µg/m³),NO2 (µg/m³),NOx (µg/m³),O3 (µg/m³),PM10 (µg/m³),PM2.5 (µg/m³),SO2 (µg/m³)
0,2019-04-02 01:00:00,4.0,Sant Marti,2.2045,41.4039,0.0,0.0,1.0,22.0,22.0,0.0,19.0,0.0,0.0
1,2019-04-02 01:00:00,42.0,Sants-Montjuic,2.1331,41.3788,0.0,0.0,2.0,29.0,32.0,0.0,0.0,0.0,0.0
2,2019-04-02 01:00:00,43.0,Eixample,2.1538,41.3853,0.0,0.2,3.0,27.0,32.0,75.0,15.0,0.0,1.0
3,2019-04-02 01:00:00,44.0,Gracia,2.1534,41.3987,0.0,0.3,13.0,56.0,76.0,44.0,21.0,0.0,1.0
4,2019-04-02 01:00:00,50.0,Ciutat Vella,2.1874,41.3864,0.0,0.0,1.0,32.0,32.0,64.0,0.0,0.0,0.0


In [17]:
# Convert timestamps to datetime and floor to hourly frequency
df_inhaler['timestamp'] = pd.to_datetime(df_inhaler['timestamp']).dt.floor('h')
df_air_quality['timestamp'] = pd.to_datetime(df_air_quality['timestamp']).dt.floor('h')

# Aggregate air quality data by district and timestamp
air_agg = df_air_quality.groupby(['district', 'timestamp']).agg({
    'NO2 (µg/m³)': 'mean',
    'PM2.5 (µg/m³)': 'mean',
    'O3 (µg/m³)': 'mean'
}).reset_index()

# Merge the data
merged_df = pd.merge(
    df_inhaler[['timestamp', 'patient_id', 'puffs', 'age', 'gender', 'smoker', 'district']],
    air_agg,
    on=['district', 'timestamp'],
    how='left'
)

# Handle missing values (using forward fill for temporal gaps)
merged_df[['NO2 (µg/m³)', 'PM2.5 (µg/m³)', 'O3 (µg/m³)']] = merged_df.groupby('district')[
    ['NO2 (µg/m³)', 'PM2.5 (µg/m³)', 'O3 (µg/m³)']
].ffill()

# Select and order final columns
df_final = merged_df[[
    'timestamp',
    'district',
    'patient_id',
    'puffs', 
    'age',
    'gender',
    'smoker',
    'NO2 (µg/m³)',
    'PM2.5 (µg/m³)',
    'O3 (µg/m³)'
]]

# Show sample result
df_final.to_parquet("../data/processed/inhaler_air_merged/", engine='pyarrow', partition_cols=['patient_id'])